## Imports

In [1]:
import os
import pathlib

import numpy as np
import pandas as pd
import sqlalchemy as sa
import statsmodels.api as sm
import statsmodels.stats.api as sms
from dotenv import dotenv_values
from scipy.stats import ttest_ind
from statsmodels.stats.power import tt_ind_solve_power
from tqdm import tqdm
from datetime import datetime, timedelta

In [2]:
os.environ["OMP_NUM_THREADS"] = "8"
os.environ["MKL_NUM_THREADS"] = "8"
os.environ["OPENBLAS_NUM_THREADS"] = "8"

In [3]:
config = dotenv_values("/home/jovyan/.env") 

def get_query_clickhouse(q: str) -> pd.DataFrame:
    """
    Function to import credentials and run query
    """
    ch_host = config['CH_HOST']
    ch_cert = config['CH_CERT']
    ch_port = config['CH_PORT']
    ch_db   = config['CH_READ_DB']
    ch_user = config['CH_READ_USER']
    ch_pass = config['CH_READ_PASS']
    
    engine = sa.create_engine(
        f"clickhouse+native://{ch_user}:"
        f"{ch_pass}@{ch_host}:"
        f"{ch_port}/{ch_db}?secure=True"
    )
    return pd.read_sql_query(q, con=engine)

## Funcs

In [4]:
def get_MDE(metric, exp_ios_data, obs_ios_data, total_frac=1, test_fraction=0.5, alpha=0.05, power=0.8, optional=False):   
    print(f"Целевая метрика - {metric}")

    alpha = alpha # alpha = 0.05, для которой считаем MDE
    power = power # мощность = 0.8, для которой считаем MDE
    test_fraction = test_fraction # Тест-контроль 50 на 50
    exp_gmv_data = exp_ios_data.copy()
    obs_gmv_data = obs_ios_data.copy().rename({metric: "CUPED_X"}, axis=1)

    gmv_data = exp_gmv_data[["anonymous_id", metric]].merge(obs_gmv_data[["anonymous_id", "CUPED_X"]], how="left", on="anonymous_id")
    gmv_data = gmv_data.sample(frac=total_frac)

    # обработаем юзеров без истории
    gmv_data["missing_CUPED"] = 0
    gmv_data.loc[gmv_data["CUPED_X"].isna(), "missing_CUPED"] = 1
    gmv_data = gmv_data.fillna(0)
    if optional:
        print(f"Пользователей без истории {gmv_data.missing_CUPED.mean():.4%}")
    # в соответствие с процедурой CUPED'a случайным образом выделим тестовую и контрольную группы

    gmv_data["treatment"] = np.random.choice([0,1], size=gmv_data.shape[0])
    # применим CUPED, получив по итогу пост-cuped метрику CUPED_GMV
    y_control = gmv_data.query("treatment==0")[metric]
    X_cov_control = gmv_data.query("treatment==0")[["CUPED_X", "missing_CUPED"]]
    y_hat = sm.OLS(y_control, X_cov_control).fit().predict(gmv_data[["CUPED_X", "missing_CUPED"]])
    gmv_data["CUPED_GMV"] = gmv_data[metric] - y_hat
    # зафиксируем снижение дисперсии
    if optional:
        print(f"\nИсходная дисперсия = {gmv_data[metric].std():.8}")
        print(f"Дисперсия после cuped = {gmv_data.CUPED_GMV.std():.8}")
        print(f"Уменьшение на {1-gmv_data.CUPED_GMV.std()/gmv_data[metric].std():.3%}")
    gmv_test_observations = gmv_data.sample(frac=test_fraction)


    sd_gmv = gmv_test_observations.CUPED_GMV.std() # считаем по метрике CUPED_GMV
    est_gmv = gmv_test_observations[metric].mean() # считаем по дефолтной метрике, потому что размер эффекта не изменится

    ratio = (1-test_fraction) / (test_fraction) # пропорция контроль/тест
    lift_gmv = 0.0075 # размер эффекта в %, для которого оцениваем мощность
    nobs_test = gmv_test_observations.shape[0] # количество наблюдений (юзеров) в тесте

    effect_size_gmv = tt_ind_solve_power(power=power, nobs1=nobs_test, alpha=alpha, ratio=ratio)
    mde_gmv = effect_size_gmv * sd_gmv/est_gmv
    # в процентах
    print(f"\nMDE равно {mde_gmv:.2%}")


    eff_size_gmv = lift_gmv * est_gmv/sd_gmv
    power_gmv = tt_ind_solve_power(effect_size=eff_size_gmv, nobs1=nobs_test, alpha=alpha, ratio=ratio)
    # мощность для эффекта в 0.75% (lift_aov)
    if optional:
        print(f"Мощность теста для эффекта в {lift_gmv:.4%} равна {power_gmv:.6}")


    nobs_gmv_test = tt_ind_solve_power(effect_size=eff_size_gmv, power=power, alpha=alpha, ratio=ratio)
    # ratio = nobs_control/nobs_test
    nobs_gmv_control = nobs_gmv_test*ratio
    # всего нужно наблюдений, наблюдений в тестовой группе, в контрольной
    if optional:
        print(f"Всего необходимо наблюдений: {round(nobs_gmv_control + nobs_gmv_test, 1):,}")
        print(f"Наблюдений в тестовой группе: {round(nobs_gmv_test, 1):,}")
        print(f"Наблюдений в контрольной группе: {round(nobs_gmv_control, 1):,}")
    if optional:
        print('---------------------------------------------------')
    else:
        print('-------------------------')


In [5]:
def date_n_weeks_ago(n, param=False):
    # Get the current date
    if param:
        d=2
    else:
        d=1
    current_date = datetime.now() - timedelta(days=d)

    # Calculate the date n weeks ago
    date_n_weeks_ago = current_date - timedelta(weeks=n)

    # Format the date
    formatted_date = date_n_weeks_ago.strftime('%Y-%m-%d')

    return formatted_date


In [12]:
def collect_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            anonymous_id,
            max(if(type_delivery='pickup', 1, 0)) as ord_flg,
            max(if(shipment_state='canceled', 1, 0)) as cancel_flg
        from cdm.ab__metrics_data
        where 1=1
            and (pickup_store_selected>0 or shop_selected_pickup>0)
            and platform in ('android', 'ios')
            and toDate(date_msk) between toDate('{date_n_weeks_ago(n_weeks)}') and toDate('{date_n_weeks_ago(0)}')
        group by anonymous_id
        """
    return get_query_clickhouse(q)

def collect_cuped_data(n_weeks) -> pd.DataFrame:
    q = f"""
        select 
            anonymous_id,
            max(if(type_delivery='pickup', 1, 0)) as ord_flg,
            max(if(shipment_state='canceled', 1, 0)) as cancel_flg
        from cdm.ab__metrics_data
        where 1=1
            and (pickup_store_selected>0 or shop_selected_pickup>0)
            and platform in ('android', 'ios')
            and toDate(date_msk) between toDate('{date_n_weeks_ago(2*n_weeks, param=True)}') and toDate('{date_n_weeks_ago(n_weeks, param=True)}')   
        group by anonymous_id
        """
    return get_query_clickhouse(q)

# Отмены самовывоза из выбора магазина

In [14]:
for i in [3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    obs_ios_data = collect_cuped_data(n_weeks=i)
    for metric in ['cancel_flg']:
        get_MDE(metric, exp_ios_data, obs_ios_data)

3 недель:
Целевая метрика - cancel_flg

MDE равно 4.97%
-------------------------
4 недель:
Целевая метрика - cancel_flg

MDE равно 4.37%
-------------------------
5 недель:
Целевая метрика - cancel_flg

MDE равно 3.81%
-------------------------
6 недель:
Целевая метрика - cancel_flg

MDE равно 3.45%
-------------------------


# Сквозная в заказ из выбора магазина

In [13]:
for i in [3,4,5,6]:
    print(f'{i} недель:')
    exp_ios_data = collect_data(n_weeks=i)
    obs_ios_data = collect_cuped_data(n_weeks=i)
    for metric in ['ord_flg']:
        get_MDE(metric, exp_ios_data, obs_ios_data)

3 недель:
Целевая метрика - ord_flg

MDE равно 2.60%
-------------------------
4 недель:
Целевая метрика - ord_flg

MDE равно 2.29%
-------------------------
5 недель:
Целевая метрика - ord_flg

MDE равно 2.07%
-------------------------
6 недель:
Целевая метрика - ord_flg

MDE равно 1.92%
-------------------------
